In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import glob
from transforms import *
from reprojection import *

In [2]:
PHI_TOP = '/home/ulyanov/data/solo/phi/'

files = sorted(glob.glob(PHI_TOP + '/polar/*'))

In [3]:
file = files[100]

with fits.open(file) as hdul:
    header = hdul[0].header
    data = hdul[0].data


In [21]:
view = View.from_header(header)
transform = view.to_spherical()

grid = np.mgrid[-90:91:1,-180:180:1]
grid, _ = transform(grid)
grid = np.array(grid)

meridians = grid[:,:,::15]
parallels = np.transpose(grid[:,::15,:], (0,2,1))

plt.figure(figsize=(10,10))
plt.imshow(data, origin='lower', cmap='seismic', vmin=-40, vmax=40)
plt.plot(meridians[1], meridians[0], color='black', ls='--', lw=1, alpha=0.5)
plt.plot(parallels[1], parallels[0], color='black', ls='--', lw=1, alpha=0.5)
plt.tight_layout()

In [18]:
data_ = remap(data, header, dlat=0.2, dlon=0.2, correct_mu=True)

In [19]:
def show_map(data, figsize=(15,8), label=r'$B_{r}$, G', cmap='seismic', vmin=-50, vmax=50, **kwargs):
    import matplotlib.pyplot as plt

    nx, ny = data.shape
    dx, dy = nx // 12, ny // 12

    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(data, origin='lower', cmap=cmap, vmin=vmin, vmax=vmax, **kwargs)

    ax.set_xticks(np.arange(0, ny + 1, dy), np.arange(-180,181,30))
    ax.set_yticks(np.arange(0, nx + 1, dx), np.arange(-90,91,15))

    ax.set_xlabel('Longitude, degrees')
    ax.set_ylabel('Latitude, degrees')

    cax = ax.inset_axes((0.03, 0.1, 0.1, 0.015))
    fig.colorbar(im, cax=cax, orientation='horizontal', label=label)

    ax.grid(True, ls='--', color='black', alpha=0.2)
    plt.tight_layout()

In [20]:
show_map(data_)